In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
import random

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.decomposition import PCA
import xgboost
from sklearn import tree
import seaborn as sns
import matplotlib.pyplot as plt
import time
# Deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
SAVE_PATH = '/content/drive/MyDrive/kaggle_project_data/'

In [4]:
train = pd.read_csv(SAVE_PATH + 'train.csv', dtype={'language': str,
                                                    'song_length_cat': str,
                                                    'decade': str})
val = pd.read_csv(SAVE_PATH + 'val.csv', dtype={'language': str,
                                                'song_length_cat': str,
                                                'decade': str})
test = pd.read_csv(SAVE_PATH + 'test.csv', dtype={'genre_first':str,
                                                  'language': str,
                                                  'song_length_cat': str,
                                                  'decade': str})

In [5]:
train.shape

(5901934, 34)

In [6]:
val.shape

(1475484, 34)

In [7]:
# train = train[4401934:] # 1.5M data points
# #tr_data = train_all[3401934:] # 2.5M data points
# val = val[:500000]

# print('size of train data : ', train.shape, 'size of val data : ', val.shape)

In [8]:
# turning some categorical features into categorical ones and encoding labels
categor_ft = ['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'artist_name', 'composer',
              'lyricist', 'language', 'genre_ids', 'song_length_cat', 'name', 'decade', 'country', 'genre_first']

for feature in categor_ft:
  le = LabelEncoder()
  comb = train[feature].append(val[feature])
  comb = set(comb.append(test[feature]))
  comb = np.array(list(comb))
  le = le.fit(comb)
  train[feature] = le.transform(train[feature].values.reshape(-1,1))
  val[feature] = le.transform(val[feature].values.reshape(-1,1))
  test[feature] = le.transform(test[feature].values.reshape(-1,1))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
X_train = train.drop(['target'], axis=1)
y_train = train['target'].values

X_val = val.drop(['target'], axis=1)
X_val = X_val[['Unnamed: 0', 'msno', 'song_id', 'source_system_tab',
       'source_screen_name', 'source_type', 'user_plays', 'song_plays',
       'user_repeat_ratio', 'song_repeat_ratio', 'city', 'bd', 'gender',
       'registered_via', 'membership_days', 'artist_name', 'composer',
       'lyricist', 'language', 'genre_ids', 'song_length_cat',
       'artist_name_number', 'composer_number', 'lyricist_number',
       'genre_number', 'name', 'decade', 'country', 'genre_first',
       'source_type_ratio', 'source_system_tab_ratio', 'artist_name_ratio',
       'composer_ratio']]
y_val = val['target'].values


del train, val; gc.collect()

d_train = lgb.Dataset(X_train, y_train)
d_val = lgb.Dataset(X_val, y_val)

In [10]:
X_train.head()

,Unnamed: 0,msno,song_id,source_system_tab,source_screen_name,source_type,user_plays,song_plays,user_repeat_ratio,song_repeat_ratio,city,bd,gender,registered_via,membership_days,artist_name,composer,lyricist,language,genre_ids,song_length_cat,artist_name_number,composer_number,lyricist_number,genre_number,name,decade,country,genre_first,source_type_ratio,source_system_tab_ratio,artist_name_ratio,composer_ratio
0,0,9176,86884,1,7,6,5511,215,0.506442,0.474419,1,0,2.0,7,2103,3784,16653,24099,8,337,1,1.0,2.0,0.0,1.0,60284,10,33,104,0.642,0.604,0.667,0.500
1,1,19273,260594,3,8,4,622,1,0.742765,1.000000,13,24,0.0,9,2301,36868,69766,24099,8,99,1,1.0,0.0,0.0,1.0,95288,8,109,33,0.662,0.788,0.875,0.766
2,2,19273,140755,3,8,4,622,4,0.742765,0.500000,13,24,0.0,9,2301,24601,51540,24099,8,99,1,1.0,1.0,0.0,1.0,67086,9,109,33,0.662,0.788,1.000,1.000
3,3,19273,27577,3,8,4,622,1,0.742765,1.000000,13,24,0.0,9,2301,31651,41991,24099,0,6,1,1.0,1.0,0.0,1.0,40702,10,33,3,0.662,0.788,1.000,1.000
4,4,9176,38706,1,7,6,5511,412,0.506442,0.364078,1,0,2.0,7,2103,5190,9701,24099,8,2,1,1.0,3.0,0.0,1.0,142810,10,84,2,0.642,0.604,1.000,1.000


In [11]:
# replacing NA with 0
X_train[['artist_name_number', 'composer_number', 'lyricist_number',
       'genre_number']] = X_train[['artist_name_number', 'composer_number', 'lyricist_number',
       'genre_number']].fillna(0, inplace=False)


X_val[['artist_name_number', 'composer_number', 'lyricist_number',
       'genre_number']] = X_val[['artist_name_number', 'composer_number', 'lyricist_number',
       'genre_number']].fillna(0, inplace=False)

test[['artist_name_number', 'composer_number', 'lyricist_number',
       'genre_number', 'user_repeat_ratio', 'song_repeat_ratio']] = test[['artist_name_number', 'composer_number', 'lyricist_number',
       'genre_number', 'user_repeat_ratio', 'song_repeat_ratio']].fillna(0, inplace=False)

In [12]:
print('nan: ', X_train.columns[X_train.isin([np.nan]).any()])

nan:  Index([], dtype='object')


In [13]:
print('nan: ', X_val.columns[X_train.isin([np.nan]).any()])

nan:  Index([], dtype='object')


In [ ]:
print('nan: ', test.columns[test.isin([np.nan]).any()])

nan:  Index([], dtype='object')


### Log reg

Using GridSearchCV for hyper-parameter tuning

In [ ]:
start = time.time()
parameters = {'penalty':['l2', 'l1'], 
                'alpha':[0.01, 0.05, 0.1]}
                
clf = SGDClassifier(loss='hinge', n_jobs=-1, random_state=23, class_weight='balanced' )
model = GridSearchCV(clf, parameters, scoring = 'roc_auc', n_jobs=-1, verbose=2, cv=3)
model.fit(X_train, y_train)

print(model.best_estimator_)
print('AUC_train = ',model.score(X_train, y_train))
print('AUC_val = ',model.score(X_val, y_val))
print('Time : ', (time.time() -  start))

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


SGDClassifier(alpha=0.05, class_weight='balanced', n_jobs=-1, random_state=23)
AUC_train =  0.5628199284167301
AUC_val =  0.5711617145251036
Time :  3404.0597376823425


In [ ]:
lr = SGDClassifier(alpha=0.05, class_weight='balanced', n_jobs=-1, random_state=23)
lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


SGDClassifier(alpha=0.05, class_weight='balanced', n_jobs=-1, random_state=23)

In [ ]:
print('AUC_train = ',lr.score(X_train, y_train))
print('AUC_val = ',lr.score(X_val, y_val))

AUC_train =  0.5723286666666667
AUC_val =  0.444506


### SVM

In [ ]:
start = time.time()
parameters = {'penalty':['l2', 'l1'], 
                'alpha':[0.0001, 0.01, 0.1]}
                
clf = SGDClassifier(loss='hinge', n_jobs=-1, random_state=23, class_weight='balanced' )
model = GridSearchCV(clf, parameters, scoring = 'roc_auc', n_jobs=-1, verbose=2, cv=3)
model.fit(X_train, y_train)

print(model.best_estimator_)
print('AUC_train = ',model.score(X_train, y_train))
print('AUC_val = ',model.score(X_val, y_val))
print('Time : ', (time.time() -  start))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
SGDClassifier(class_weight='balanced', n_jobs=-1, penalty='l1', random_state=23)
AUC_train =  0.5639613039898813
AUC_val =  0.5693538726333014
Time :  4025.5501539707184


In [ ]:
X_train.shape

(5901934, 33)

In [ ]:
5901934 - 1500000

4401934

In [ ]:
X_val.shape

(1475484, 33)

### Random Forest

Problems with RAM on this large data. Using samples for those models

In [14]:
# sampling

X_train_sample = X_train[4401934:] 
y_train_sample = y_train[4401934:] 

X_val_sample = X_val[:150000]
y_val_sample = y_val[:150000]


print(X_train_sample.shape, X_val_sample.shape)

(1500000, 33) (150000, 33)


In [15]:
# start = time.time()
# parameters = {'n_estimators':[100, 300, 1000],
#               'max_depth': [15, 25, 35],
#               'min_samples_split':[3, 5, 10]
#               }
# clf = RandomForestClassifier(random_state=23, class_weight='balanced', n_jobs=-1)
# model = GridSearchCV(clf, parameters, scoring = 'roc_auc', verbose=2, cv=3)
# model.fit(X_train_sample, y_train_sample)

# print(model.best_estimator_)
# print('AUC_train = ',model.score(X_train_sample, y_train_sample))
# print('AUC_val = ',model.score(X_val_sample, y_val_sample))
# print('Time : ', (time.time() -  start))

In [21]:
start = time.time()

model = RandomForestClassifier(random_state=23, class_weight='balanced', n_jobs=-1)

model.fit(X_train_sample, y_train_sample)



RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=23)

In [22]:
print('AUC_train = ',model.score(X_train_sample, y_train_sample))
print('AUC_val = ',model.score(X_val_sample, y_val_sample))
print('Time : ', (time.time() -  start))

AUC_train =  0.999998
AUC_val =  0.6999666666666666
Time :  481.1014447212219


### Decision Tree

In [ ]:
# start = time.time()
# parameters = {'max_depth':[10, 15, 20], 
#                 'min_samples_split':[5, 10, 20],
#                 'max_leaf_nodes': [10, 50, 100]}

# clf = RandomForestClassifier(random_state=23, class_weight='balanced')
# model = GridSearchCV(clf, parameters, scoring = 'roc_auc', verbose=2, cv=3)
# model.fit(X_train_sample, y_train_sample)

# print(model.best_estimator_)
# print('AUC_train = ',model.score(X_train_sample, y_train_sample))
# print('AUC_val = ',model.score(X_val_sample, y_val_sample))
# print('Time : ', (time.time() -  start))

In [19]:
model = DecisionTreeClassifier(random_state=23, class_weight='balanced' )

model.fit(X_train_sample, y_train_sample)



DecisionTreeClassifier(class_weight='balanced', random_state=23)

In [20]:
print('AUC_train = ',model.score(X_train_sample, y_train_sample))
print('AUC_val = ',model.score(X_val_sample, y_val_sample))
print('Time : ', (time.time() -  start))

AUC_train =  1.0
AUC_val =  0.6264666666666666
Time :  635.3794229030609


### LightGBM

In [ ]:
d_train = lgb.Dataset(X_train, y_train)
d_val = lgb.Dataset(X_val, y_val)

In [ ]:
print('Training LGBM model...')
params = {}
params['learning_rate'] = 0.2
params['objective'] = 'binary'
params['min_child_samples'] = 100
params['bagging_fraction'] = 0.7
params['feature_fraction'] = 0.7
params['bagging_frequency'] = 5
params['metric'] = 'auc'

model = lgb.train(params, 
                  train_set=d_train, 
                  num_boost_round=15000, early_stopping_rounds=100, 
                  valid_sets=d_val,
                  verbose_eval=500)

Training LGBM model...
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[36]	valid_0's auc: 0.749607


the best result so far

In [ ]:
test = test.rename(columns={"Unnamed: 0": "id"})

In [ ]:
ids = test['id'].values

In [ ]:
# saving test results
def predict_and_save(clf, x_te, model):
  file_name = 'submission_' + model +'.csv.gz'
  p_test = clf.predict(x_te)
  subm = pd.DataFrame()
  subm['id'] = ids
  subm['target'] = p_test
  subm.to_csv(file_name, compression = 'gzip', index=False, float_format = '%.5f')


In [ ]:
predict_and_save(model, test, 'lgb')

### CatBoosting

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.4 MB/s 


In [ ]:
import catboost as cb
from catboost import CatBoostClassifier, Pool


In [ ]:
model_cb = cb.CatBoostClassifier(
        iterations=300, 
        learning_rate=0.1, 
        depth=6, 
        loss_function='Logloss',
        thread_count=30,
        random_seed=229,
        use_best_model = True,
        l2_leaf_reg = 2
    )

cat_features = ['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'artist_name', 'composer',
              'lyricist', 'language', 'genre_ids', 'song_length_cat', 'name', 'decade', 'country', 'genre_first']
eval_dataset = Pool(X_val,y_val,cat_features=cat_features)

model_cat = model_cb.fit(X_train, y_train,cat_features,eval_set=eval_dataset,verbose_eval=25,early_stopping_rounds = 100)

0:	learn: 0.4628597	test: 0.6575037	best: 0.6575037 (0)	total: 29.2s	remaining: 2h 25m 25s


In [ ]:
model_cb, y_val_pred = run_cb(X_train, y_train['target'], X_val, y_val['target'],cat_features)



In [ ]:
print(roc_auc_score(y_val['target'],y_val_pred))

### PCA

In [ ]:
len(X_train.keys())

33

In [ ]:
pca = PCA(n_components=33)

In [ ]:
pca.fit(X_train)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('№ of components')
plt.ylabel('Cumul. explained variance')

In [ ]:
Pca = PCA(n_components=10)
pca = Pca.fit(X_train)
X_train_pca =  pca.transform(X_train)
X_val_pca = pca.transform(X_val)
test_pca = pca.transform(test)

In [ ]:
X_train_pca.shape

In [ ]:
del X_train, X_val, test

### LightGBM_PCA

In [ ]:
d_train = lgb.Dataset(X_train_pca, y_train)
d_val = lgb.Dataset(X_val, y_val)

print('Training LGBM model...')
params = {}
params['learning_rate'] = 0.2
params['objective'] = 'binary'
params['min_child_samples'] = 100
params['bagging_fraction'] = 0.7
params['feature_fraction'] = 0.7
params['bagging_frequency'] = 5
params['metric'] = 'auc'

model = lgb.train(params, 
                  train_set=d_train, 
                  num_boost_round=15000, early_stopping_rounds=100, 
                  valid_sets=d_val,
                  verbose_eval=500)




In [ ]:
test = test.rename(columns={"Unnamed: 0": "id"})

ids = test['id'].values

def predict_and_save(clf, x_te, model):
  file_name = 'submission_' + model +'.csv.gz'
  p_test = clf.predict(x_te)
  subm = pd.DataFrame()
  subm['id'] = ids
  subm['target'] = p_test
  subm.to_csv(file_name, compression = 'gzip', index=False, float_format = '%.5f')

predict_and_save(model, test, 'lgb')

In [ ]:
# start = time.time()
# parameters = { 'learning_rate' : [0.05, 0.10, 0.15],
#               'max_depth': [10, 12, 15],
#               'min_child_weight' : [3, 5, 7]}

# clf = xgboost.XGBClassifier(objective='binary:logistic', random_state=2)
# model = GridSearchCV(clf, parameters, scoring = 'roc_auc', verbose=2, cv=3)
# model.fit(X_train_pca, y_train)

# print(model.best_estimator_)
# print('AUC_train = ',model.score(X_train_pca, y_train))
# print('AUC_val = ',model.score(X_val_pca, y_val))
# print('Time : ', (time.time() -  start))

In [ ]:
xgb = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.3,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

xgb.fit(X_train_pca, y_train)

[12:51:33] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBClassifier(colsample_bytree=0.5, gamma=0.3, learning_rate=0.05, max_depth=15,
              min_child_weight=7, random_state=2)

In [ ]:
xgb = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.3,
              learning_rate=0.05, max_delta_step=0, max_depth=15)

xgb.fit(X_train_pca, y_train)

In [ ]:
print('AUC_train = ',xgb.score(X_train_pca, y_train))
print('AUC_val = ',xgb.score(X_val_pca, y_val))

AUC_train =  0.6104041488773002
AUC_val =  0.4096181320841161
